In [2]:
import numpy as np
import pandas as pd
from geopy.distance import distance
X = np.load('x_train.npy')
y = np.load('y_train.npy')
print(X.shape)

X = X[y < 3000000]
y = y[y < 3000000]
print(X.shape)

y = y[X['sqft_lot'] < 300000]
X = X[X['sqft_lot'] < 300000]
print(X.shape)

y = y[X['bedrooms'] < 15]
X = X[X['bedrooms'] < 15]
print(X.shape)

_pred = (X['condition'] > 2) | (y < 1000000)
y = y[_pred]
X = X[_pred]
print(X.shape)

y = y[X['long'] < -121.5]
X = X[X['long'] < -121.5]
print(X.shape)

X = pd.DataFrame(X)

def spherical_dist(pos1, pos2, r=3958.75):
    pos1 = pos1 * np.pi / 180
    pos2 = pos2 * np.pi / 180
    cos_lat1 = np.cos(pos1[..., 0])
    cos_lat2 = np.cos(pos2[..., 0])
    cos_lat_d = np.cos(pos1[..., 0] - pos2[..., 0])
    cos_lon_d = np.cos(pos1[..., 1] - pos2[..., 1])
    return r * np.arccos(cos_lat_d - cos_lat1 * cos_lat2 * (1 - cos_lon_d))

X['y'] = y
X['50nn'] = 0
for idx in X.index:
    if idx % 500 == 0:
        print(idx)
    coord = np.array([X['lat'][idx], X['long'][idx]])
    dist = pd.DataFrame(spherical_dist(X[['lat', 'long']].as_matrix(), coord))
    dist[1] = X['y']
    dist = dist[dist[0] != 0].sort_values(by=[0,])[0:50]
    X['50nn'][idx] = (dist[1] / dist[0]).sum()

_X = X.copy()
    
X = X.drop(columns=['date', 'yr_renovated', 'sqft_above'])
X.loc[X['sqft_basement'] == 0, 'sqft_basement'] = X[X['sqft_basement'] != 0]['sqft_basement'].mean()
#X.loc[X['yr_renovated'] == 0, 'yr_renovated'] = X[X['yr_renovated'] != 0]['yr_renovated'].mean()
X.loc[:, 'zipcode']  = X['zipcode'] % 1000
X.loc[:, 'sqft_lot']  = np.log(X['sqft_lot'])


zips = X.groupby(by=['zipcode'])['y'].mean().sort_values()
zipmap = dict()
for i in range(0, 66):
    zipmap[zips.index[i]] = i
for i in range(0, 4):
    zipmap[zips.index[66 + i]] = 65 + (i + 1)*10
X.loc[:, 'zipcode'] = [zipmap[z] for z in X['zipcode']]


    
y = np.log(y)
X = X.drop(columns=['y',])

X['bias'] = 1

(1000,)
(1000,)
(995,)
(995,)
(994,)
(993,)
0


/usr/lib/python3/dist-packages/ipykernel_launcher.py:48: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/usr/lib/python3/dist-packages/ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


500


In [3]:
%pylab inline

import seaborn as sns
sns.set(style="whitegrid")

from sklearn import datasets
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge, Lasso, SGDRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PolynomialFeatures
import pandas as pd
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
X = X.drop(columns=['lat', 'long'])
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size=0.5, random_state=42)

Populating the interactive namespace from numpy and matplotlib


/usr/lib/python3/dist-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['dist']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [5]:
param_grid = {"alpha": np.logspace(-6, 6)}
gsl1 = GridSearchCV(Lasso(max_iter=10000), param_grid=param_grid)
pf = PolynomialFeatures(2)
ss = MinMaxScaler()
ss.fit(X)
allX = pd.DataFrame(pf.fit_transform(ss.transform(X)))
allX['bias'] = 1
gsl1.fit(allX, y)

GridSearchCV(cv=None, error_score='raise',
       estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=10000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'alpha': array([1.00000e-06, 1.75751e-06, 3.08884e-06, 5.42868e-06, 9.54095e-06,
       1.67683e-05, 2.94705e-05, 5.17947e-05, 9.10298e-05, 1.59986e-04,
       2.81177e-04, 4.94171e-04, 8.68511e-04, 1.52642e-03, 2.68270e-03,
       4.71487e-03, 8.28643e-03, 1.45635e-02, 2.55955e-02, 4.49... 3.39322e+04, 5.96362e+04,
       1.04811e+05, 1.84207e+05, 3.23746e+05, 5.68987e+05, 1.00000e+06])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [6]:
gsl1.best_score_

0.8548656539246523

In [7]:
x_test = np.load('x_test.npy')

In [8]:
x_test = pd.DataFrame(x_test)

def spherical_dist(pos1, pos2, r=3958.75):
    pos1 = pos1 * np.pi / 180
    pos2 = pos2 * np.pi / 180
    cos_lat1 = np.cos(pos1[..., 0])
    cos_lat2 = np.cos(pos2[..., 0])
    cos_lat_d = np.cos(pos1[..., 0] - pos2[..., 0])
    cos_lon_d = np.cos(pos1[..., 1] - pos2[..., 1])
    return r * np.arccos(cos_lat_d - cos_lat1 * cos_lat2 * (1 - cos_lon_d))

_X['y'] = np.exp(y)
x_test['50nn'] = 0
for idx in x_test.index:
    if idx % 500 == 0:
        print(idx)
    coord = np.array([x_test['lat'][idx], x_test['long'][idx]])
    dist = pd.DataFrame(spherical_dist(_X[['lat', 'long']].as_matrix(), coord))
    dist[1] = _X['y']
    dist = dist[dist[0] != 0].sort_values(by=[0,])[0:50]
    x_test['50nn'][idx] = (dist[1] / dist[0]).sum()

#_X = X.copy()
    
x_test = x_test.drop(columns=['date', 'yr_renovated', 'sqft_above'])
x_test.loc[x_test['sqft_basement'] == 0, 'sqft_basement'] = x_test[x_test['sqft_basement'] != 0]['sqft_basement'].mean()
#X.loc[X['yr_renovated'] == 0, 'yr_renovated'] = X[X['yr_renovated'] != 0]['yr_renovated'].mean()
x_test.loc[:, 'zipcode']  = x_test['zipcode'] % 1000
x_test.loc[:, 'sqft_lot']  = np.log(x_test['sqft_lot'])


#zips = X.groupby(by=['zipcode'])['y'].mean().sort_values()
#zipmap = dict()
#for i in range(0, 66):
#    zipmap[zips.index[i]] = i
#for i in range(0, 4):
#    zipmap[zips.index[66 + i]] = 65 + (i + 1)*10
x_test.loc[:, 'zipcode'] = [zipmap[z] for z in x_test['zipcode']]


    
#y = np.log(y)
#X = X.drop(columns=['y',])

0


In [9]:
_x_test = x_test.copy()
x_test = x_test.drop(columns=['lat', 'long'])
x_test

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,condition,grade,sqft_basement,yr_built,zipcode,50nn
0,3.0,2.00,1600,8.902456,1.0,False,4,7,810.238095,1979,55,13562511
1,4.0,3.00,3060,10.819818,1.0,False,4,8,600.000000,1957,39,14597198
2,5.0,2.50,3490,9.844268,1.0,False,4,9,1650.000000,1958,62,16610138
3,3.0,2.50,2360,9.105535,1.0,False,5,7,1020.000000,1929,21,17678589
4,4.0,3.00,2860,9.239122,1.0,False,4,8,810.238095,1953,95,15219909
5,4.0,1.00,1070,8.918249,1.5,False,3,7,810.238095,1944,8,9298937
6,4.0,2.25,2860,8.640826,2.0,False,3,10,730.000000,1933,60,12256688
7,3.0,2.00,1320,8.699515,1.5,False,4,7,810.238095,1920,48,11040989
8,3.0,1.75,1850,8.699515,1.5,False,3,8,200.000000,1913,60,16096117
9,3.0,2.00,1160,8.143517,1.0,False,4,6,580.000000,1930,36,10712703


In [10]:
x_test['bias'] = 1
x_test_ = pd.DataFrame(pf.fit_transform(ss.transform(x_test)))
x_test_['bias'] = 1

In [11]:
pred = gsl1.predict(x_test_)

In [12]:
pred = np.exp(pred)

In [13]:
pred

array([ 480825.5996755 ,  788652.74941327,  917210.04178484,
        615032.01407924, 1067822.8473951 ,  237665.76096099,
       1044138.0294778 ,  465120.63617744,  798118.27106998,
        334398.39105659,  307427.88807779,  449654.86699069,
        748528.71435915,  384556.42039138,  497344.27901615,
        633551.50250326,  912037.54253146,  663035.94938541,
        640886.8879025 ,  367038.85651191,  346366.09181724,
        334733.00602902,  505178.97744579,  593375.42401377,
        574754.43064272,  565269.29743171,  521392.67029907,
        375074.01121662,  257922.34701707,  320856.56588073,
        530992.01629986,  402226.16321043,  638013.2055971 ,
        722598.58964296,  527635.58111821,  603869.62169393,
        463448.62959421,  692404.88842473,  761190.86714162,
        304705.4729982 ,  187635.55951857,  271742.00065994,
        257263.15409813,  139538.60060535,  228998.29648774,
        553849.00886124,  339491.25687696,  364933.78896513,
        274868.89973017,

In [20]:
answ = pd.DataFrame()
answ['Id'] = range(1, len(pred) + 1)
answ['Price'] = pred

In [22]:
answ.to_csv('answ.csv',index=False)